In [15]:
import sys
sys.path.insert(0, '../gofher')

import os
import matplotlib.image as mpimg

from gofher import run_gofher
from visualize import visualize
from spin_parity import read_spin_parity_galaxies_label_from_csv

In [16]:
survery_to_use = "panstarrs"

BANDS_IN_ORDER = ['r','i','z','y']
REF_BANDS_IN_ORDER = ['i','z','y','r']

In [17]:
figure_to_run_on = "figure10"

In [18]:
path_to_catalog_data = "..\\..\\..\\spin-parity-catalog-data"
path_to_output = "..\\..\\..\\..\\"

In [19]:
def get_fits_path(name,band):
    """the file path of where existing fits files can be found"""
    return os.path.join(path_to_catalog_data,survery_to_use,figure_to_run_on,name,"{}_{}.fits".format(name,band))

def get_color_image_path(name):
    return os.path.join(path_to_catalog_data,survery_to_use,figure_to_run_on,name,"{}_color.jfif".format(name))

def get_path_to_catalog_csv():
    return os.path.join(path_to_catalog_data,"catalog","{}.csv".format(figure_to_run_on))

In [20]:
def get_paper_dark_side_labels():
    return read_spin_parity_galaxies_label_from_csv(get_path_to_catalog_csv())

def get_galaxies():
    return os.listdir(os.path.join(path_to_catalog_data,survery_to_use,figure_to_run_on))

In [21]:
paper_labels = get_paper_dark_side_labels()

print(os.listdir(path_to_output))

for name in get_galaxies():
    if name not in paper_labels: continue
    
    paper_label = paper_labels[name]
    gal = run_gofher(name,get_fits_path,['g','r','i','z','y'],['i','z','y','r'], paper_label)
    (ebm_label, ebm_p) = gal.run_ebm(['r','i','z','y'])
    print(name,paper_label,ebm_label,ebm_p)

    #(header,row) = gal.get_verbose_csv_header_and_row(['r','i','z','y'],paper_label)
    #color_image = color = mpimg.imread(get_color_image_path(name))
    #visualize(gal,color_image,['r','i','z','y'],paper_label)
    
    break


['.ipynb_checkpoints', '03_17_2023_research', '03_17_2023_research_not_seperate', '10.21105.joss.00058.pdf', '1237648702966988820_g.fits', '1237664094507237473.fits', '1706.07717.pdf', '1806.10630.pdf', '1x', '2022-Fall-Cora-Schallock-GAMBIT.pdf', '2022_fall', '2023_spring_end_of_quarter', '2024_winter', '2202.13493.pdf', '3.PNG', '3950312.pdf', '8_6_2_solution.PNG', 'adv_invite', 'adv_lecture', 'adv_payment.pdf', 'analyze_ks_d_geo.py', 'anthony_error.PNG', 'ASPlectures_Galaxies_Virtual_Sep2020_Part1_v2.pdf', 'AvisFV 3.0.lnk', 'awkcel_out.tar.gz', 'back_to_the_drawing_board', 'Baggett_1998_AJ_116_1626.pdf', 'berkley_dust_lecture_1.pdf', 'berkley_dust_lecture_2.pdf', 'blur_image.py', 'byun.pdf', 'Capture.PNG', 'Capture_0.PNG', 'center_test', 'check_for_missing_spin_parity_catalog_galaxies.py', 'check_it.fits', 'code_to_upload', 'color_traveling_salesmen.pdf', 'command_for_anthony.txt', 'comparison', 'complete_blood_work_11_08_2022.pdf', 'comp_metabolic_work_11_08_2022.pdf', 'cool.PNG', 